# RoughXP Analysis

### Author : Matthieu Fraticelli, Emma Ducos - 2023

This code is designed to analyze the data of the RoughXP experiment using EEG signals recorded from BrainProducts R-Net and MyBrainTech Q+


----

*The present code is designed to convert files recorded in XDF with LSL LabRecorder to FIF files usable in MNE.*

---

### Import required librairies

In [1]:
#To import MNE
import mne

#Other librairies needed
import pyxdf
import os
import matplotlib.pyplot as plt
import numpy as np
from mne_bids import BIDSPath, write_raw_bids

In [2]:
#To check the system of MNE
mne.sys_info()

Platform             Windows-10-10.0.19045-SP0
Python               3.10.11 | packaged by Anaconda, Inc. | (main, Apr 20 2023, 18:56:50) [MSC v.1916 64 bit (AMD64)]
Executable           C:\Users\mfratice\AppData\Local\anaconda3\envs\IDA\python.exe
CPU                  Intel64 Family 6 Model 154 Stepping 3, GenuineIntel (20 cores)
Memory               31.7 GB

Core
├☑ mne               1.4.0
├☑ numpy             1.23.5 (MKL 2021.4-Product with 14 threads)
├☑ scipy             1.10.1
├☑ matplotlib        3.5.3 (backend=module://matplotlib_inline.backend_inline)
├☑ pooch             1.7.0
└☑ jinja2            3.1.2

Numerical (optional)
├☑ sklearn           1.2.2
├☑ numba             0.56.4
├☑ nibabel           5.1.0
├☑ nilearn           0.10.0
├☑ dipy              1.6.0
├☑ pandas            2.0.0
└☐ unavailable       openmeeg, cupy

Visualization (optional)
├☑ pyvista           0.38.5 (OpenGL 4.5.0 - Build 31.0.101.3959 via Intel(R) Iris(R) Xe Graphics)
├☑ pyvistaqt         0.0.0
├☑ ipyv

---


In [25]:
sub = 'DURJIL'
ses = '20230517'
task = 'Default'
acq = 'RNET'
run = '001'

In [26]:
project_path = '/mnt/c/Users/mfratice/OneDrive - Institut Pasteur Paris/roughxpmbt/'

In [27]:
results_path = os.path.join(project_path, '04_roughxpmbt_results')
root_bids = os.path.join(project_path, '01_roughxpmbt_data/02_roughxpmbt_bids')
file_path = os.path.join(project_path, '01_roughxpmbt_data/05_roughxpmbt_labrecorder/for_matthieu/sub-{sub}/ses-{ses}/eeg/sub-{sub}_ses-{ses}_task-{task}_acq-{acq}_run-{run}_eeg.xdf'.format(sub=sub, ses=ses, task=task, acq=acq, run=run))

In [28]:
data, header = pyxdf.load_xdf(file_path)

for i, stream in enumerate(data):
    
    if stream['info']['name'][0] == 'Q+': # if the stream comes from mybraintech Q+ system
        qplus_stream = stream
    elif stream['info']['name'][0] == 'actiCHampMarkers-20110423': # if the stream comes from brainproducts actichamp system of IDA 
        actichampmarkers_stream = stream
    elif stream['info']['name'][0] == 'actiCHamp-20110423': # if the stream comes from brainproducts actichamp system of IDA 
        actichamp_stream = stream
    else:
        print("stream source unrecognized")

# actichamp data in a mne raw data format
actichamp_data = actichamp_stream["time_series"].T
actichamp_sfreq = float(actichamp_stream["info"]["nominal_srate"][0])
# actichamp_info_channels = actichamp_stream["info"]["desc"][0]["channels"][0]["channel"]
actichamp_channel_names = [channel["label"][0] for channel in actichamp_stream["info"]["desc"][0]["channels"][0]["channel"]]
actichamp_channel_types = [channel["type"][0] for channel in actichamp_stream["info"]["desc"][0]["channels"][0]["channel"]]
actichamp_channel_units = [channel["unit"][0] for channel in actichamp_stream["info"]["desc"][0]["channels"][0]["channel"]]
actichamp_info = mne.create_info(ch_names=actichamp_channel_names, sfreq=actichamp_sfreq)#, ch_types=actichamp_channel_types, verbose=True)
actichamp_raw = mne.io.RawArray(actichamp_data, actichamp_info)
actichamp_raw.plot(block=True)

# qplus data in a mne raw data format
qplus_data = qplus_stream["time_series"].T
qplus_sfreq = float(qplus_stream["info"]["nominal_srate"][0])
# qplus_info_channels = qplus_stream["info"]["desc"][0]["channels"][0]["channel"]
qplus_channel_names = [channel["label"][0] for channel in qplus_stream["info"]["desc"][0]["channels"][0]["channel"]]
qplus_channel_types = [channel["type"][0] for channel in qplus_stream["info"]["desc"][0]["channels"][0]["channel"]]
qplus_channel_units = [channel["unit"][0] for channel in qplus_stream["info"]["desc"][0]["channels"][0]["channel"]]
qplus_info = mne.create_info(ch_names=qplus_channel_names, sfreq=qplus_sfreq)#, ch_types=actichamp_channel_types, verbose=True)
qplus_raw = mne.io.RawArray(qplus_data, qplus_info)
qplus_raw.plot(block=True)



acq = 'acticap'
filename = f"sub-{sub}_ses-{ses}_task-{task}_acq-{acq}_run-{run}_eeg.fif"
# save as .fif 
fname = os.path.join(results_path, filename)
actichamp_raw.save(fname=fname, overwrite=True)

acq = 'qplus'
filename = f"sub-{sub}_ses-{ses}_task-{task}_acq-{acq}_run-{run}_eeg.fif"
# save as .fif 
fname = os.path.join(results_path, filename)
actichamp_raw.save(fname=fname, overwrite=True)



# bids_path = BIDSPath(subject=sub,
#              task=task,
#              root=root_bids)
    
# write_raw_bids(actichamp_raw, bids_path, allow_preload=True, format='FIF')

# write_raw_bids(qplus_raw, bids_path, allow_preload=True, format='FIF')#, overwrite=True)


Exception: file c\Users\mfratice\OneDrive - Institut Pasteur Paris\roughxpmbt\01_roughxpmbt_data\05_roughxpmbt_labrecorder\for_matthieu\sub-DURJIL\ses-20230517\eeg\sub-DURJIL_ses-20230517_task-Default_acq-RNET_run-001_eeg.xdf does not exist.

300Hz sampling for Q+ and 1000 for Rnet, comparer SNR, sensibilite, comparer des ERPs entre les deux pour le fond du stage. Luc parle de l'aspect autiste, sortir historgramme autisme. ERPs par casque, comparer pour ERPs et ASSRs 40Hz. Benchmark et autisme, median split, plot score aversion en fonction groupe autisme et courbes individuelles, color selon les deux codes.Correlation score stress et aversion pour voir si ca arrive. (Aversion, autisme, EEG classique, Benchmark) Cz et voisins

correlation between stai t and QA